#Pruebas de Optimización Inicial

Se muestran las primeras pruebas de optimización para el cálculo de pozos, fuentes y componentes disjuntos. Se aplican varias optimizaciones propuestas en http://docs.julialang.org/en/release-0.4/manual/performance-tips/

###Datos y cargas iniciales

In [2]:
cd("/Users/Nes/Desktop/NeuroCiencias/JuliaExperiments/Parallel-Neuroscience/ComponentesDisjuntos");

In [3]:
include("funcionesCentrosDeMasa.jl");

In [4]:
include("cargaYFiltra.jl");

INFO: Loading help data...


##Memory Order

El orden de acceso a memoria es muy importante en la velocidad y debe tomarse en cuenta cómo se recorre la información puesto que no es lo mismo acceder a memoria contigua que hacer saltos. Julia guarda la información por columnas. Por ejemplo x = [1 2;3 4] se almacena como [1 3 2 4]

In [13]:
include("memoryOrderTest.jl");

In [14]:
memoryTestRowOrder(CSD);

In [15]:
memoryTestColOrder(CSD);

In [16]:
fmt(f) = println(rpad(string(f)*": ", 14, ' '), @elapsed f(CSD))

fmt (generic function with 1 method)

In [45]:
map(fmt, Any[memoryTestRowOrder, memoryTestColOrder]);

memoryTestRowOrder: 0.001015006
memoryTestColOrder: 0.000871196


La diferencia es pequeña porque la prueba es en una matriz de 64x64. En este caso resultó ser 16% más rápido, pero haciendo varias pruebas varía entre el 8 y el 30%. Sin embargo, entre más grande es el arreglo la diferencia es más notoria y, al hacerlo en la matriz de 64x64x5000 se notará mas.

Si observamos, por ejemplo, una matriz de 1000 x 1000 basándonos en el ejemplo.

In [46]:
function copy_col_row{T}(x::Vector{T})
    n = size(x, 1)
    out = Array(T, n, n)
    for col=1:n, row=1:n
        out[row, col] = x[row]
    end
    out
end

function copy_row_col{T}(x::Vector{T})
    n = size(x, 1)
    out = Array(T, n, n)
    for row=1:n, col=1:n
        out[row, col] = x[col]
    end
    out
end

copy_row_col (generic function with 1 method)

In [47]:
x = randn(10000);

In [48]:
fmt(f) = println(rpad(string(f)*": ", 14, ' '), @elapsed f(x))

fmt (generic function with 1 method)

In [49]:
map(fmt, Any[copy_col_row, copy_row_col]);

copy_col_row: 0.687623436
copy_row_col: 1.91148332
